In [ ]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

In [ ]:
# Parameters
learning_rate = 0.01
training_epochs = 1000
display_step = 100

In [ ]:
mat_workspace = sio.loadmat('images/lena.mat')

In [ ]:
lena_im = np.array(mat_workspace['im'])
plt.imshow(lena_im)
plt.show()

In [ ]:
lena_img = np.array(mat_workspace['img']) # Grayscale
plt.imshow(lena_img, cmap='gray')
plt.show()

In [ ]:
lena_imgrad = np.array(mat_workspace['imgrad']) # Vertical Edges
plt.imshow(lena_imgrad, cmap='gray')
plt.show()

In [ ]:
mat_workspace = sio.loadmat('images/cameraman.mat')
cam_im = np.array(mat_workspace['im'])
plt.imshow(cam_im, cmap='gray')
plt.show()

In [ ]:
# Rearrange input/output data 
train_In = np.expand_dims(lena_img, axis=0)
train_In = np.expand_dims(train_In, axis=3)
train_Out = np.expand_dims(lena_imgrad, axis=0)
train_Out = np.expand_dims(train_Out, axis=3)
cam_In = np.expand_dims(cam_im, axis=0)
cam_In = np.expand_dims(cam_In, axis=3)

In [ ]:
# tf input via placeholder
In = tf.placeholder("float32", shape=[1, 512, 512, 1])
Out = tf.placeholder("float32", shape=[1, 512, 512, 1])

In [ ]:
Gx = tf.get_variable("Gx", initializer=tf.truncated_normal(shape=[3, 3, 1, 1]))
Gy = tf.get_variable("Gy", initializer=tf.truncated_normal(shape=[3, 3, 1, 1]))
Outx = tf.nn.conv2d(In, Gx, strides=[1, 1, 1, 1], padding='SAME', name="Outx")
Outy = tf.nn.conv2d(In, Gy, strides=[1, 1, 1, 1], padding='SAME', name="Outy")

In [ ]:
# Pred = G = sqrt(Edgex^2 + Edgey^2)
Pred = tf.sqrt(tf.add(tf.square(Outx), tf.square(Outy)))

In [ ]:
# Cost function - Absolute Difference
cost = tf.losses.absolute_difference(labels=Out, predictions=Pred)

In [ ]:
# Optimizer Adam
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
# Initialize the variables
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    # Fit all training data
    for epoch in range(training_epochs):
        c, out, _ = sess.run([cost, Pred, optimizer],
                             feed_dict={In: train_In, Out: train_Out})

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch), "cost=", "{:.9f}".format(c))
            print('Lena')
            plt.imshow(np.squeeze(out), cmap='gray')
            plt.show()
            print('Cameraman')
            out = sess.run(Pred, feed_dict={In:cam_In})
            plt.imshow(np.squeeze(out), cmap='gray')
            plt.show()
    out = sess.run(Pred, feed_dict={In:cam_In})
    out = np.squeeze(out)
    mpimg.imsave("out.png", out, cmap='gray')
    mpimg.imsave("cameraman.png", cam_im, cmap='gray')
